In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from collections import defaultdict

import numpy as np


from scipy.special import factorial
from tqdm import tqdm
from collections import Counter

import math

In [3]:

def read_graphml(graphml_file):
    try:
        # Read the GraphML file
        G = nx.read_graphml(graphml_file)
        print(f"Graph loaded successfully:")
        print(f"Number of nodes: {G.number_of_nodes()}")
        print(f"Number of edges: {G.number_of_edges()}")
        return G
    except Exception as e:
        print(f"Error reading GraphML file: {str(e)}")
        return None

A small-world network is a graph characterized by a high clustering coefficient and low distances



To determine if a network exhibits small-world properties, you typically look for two key characteristics:

**High Clustering Coefficient:** Small-world networks tend to have a significantly higher clustering coefficient than would be expected in a random graph of the same size and average degree.


**Short Average Path Length:** Despite the high clustering, small-world networks also have short average path lengths, similar to or even shorter than those in random graphs.

In [4]:
G_morpho = read_graphml("morphological_parrot_network.graphml")

Graph loaded successfully:
Number of nodes: 398
Number of edges: 1372


In [25]:
def network_metrics(G):
    # Average path length
    try:
        avg_path = nx.average_shortest_path_length(G)
    except nx.NetworkXError:  # handles disconnected graphs
        avg_path = float('inf')

    # Global clustering coefficient (transitivity)
    clustering = nx.transitivity(G)

    print(f"Network Metrics:")
    print("-" * 30)
    print(f"Average Path Length: {avg_path:.3f}")
    print(f"Global Clustering Coefficient: {clustering:.3f}")

    return avg_path, clustering

In [12]:
network_metrics(G_morpho)

Network Metrics:
------------------------------
Average Path Length: 6.536
Global Clustering Coefficient: 0.430


In [14]:
def create_degree_preserved_random(G):
    # Get the degree sequence of original network
    degree_seq = [d for n, d in G.degree()]

    # Create random graph with same degree sequence
    random_G = nx.configuration_model(degree_seq, seed=42)

    # Remove parallel edges and self-loops
    random_G = nx.Graph(random_G)

    return random_G

In [20]:
G_rand = create_degree_preserved_random(G_morpho)

In [21]:
network_metrics(G_rand)

Network Metrics:
------------------------------
Average Path Length: 3.339
Global Clustering Coefficient: 0.016


In [26]:
def is_small_world(G, n_random=100):
    # Original network metrics
    orig_path, orig_clust = network_metrics(G)

    # Generate random networks and compute average metrics
    random_paths = []
    random_clusts = []

    for i in range(n_random):
        # Create random network preserving degree sequence
        random_G = create_degree_preserved_random(G)

        # Calculate metrics
        rand_path, rand_clust = network_metrics(random_G)
        random_paths.append(rand_path)
        random_clusts.append(rand_clust)

    # Average random metrics
    avg_rand_path = np.mean(random_paths)
    avg_rand_clust = np.mean(random_clusts)

    print("\nSmall-World Analysis:")
    print("-" * 30)
    print(f"Original Network:")
    print(f"  Clustering: {orig_clust:.3f}")
    print(f"  Path Length: {orig_path:.3f}")
    print(f"\nRandom Networks (average over {n_random} networks):")
    print(f"  Clustering: {avg_rand_clust:.3f}")
    print(f"  Path Length: {avg_rand_path:.3f}")

    # Check small-world properties
    is_sw = (orig_clust > avg_rand_clust) and (orig_path <= avg_rand_path * 1.5)

    print(f"\nSmall-world properties: {'Present' if is_sw else 'Absent'}")

    return is_sw

In [27]:
is_small_world(G_morpho)

Network Metrics:
------------------------------
Average Path Length: 6.536
Global Clustering Coefficient: 0.430
Network Metrics:
------------------------------
Average Path Length: 3.339
Global Clustering Coefficient: 0.016
Network Metrics:
------------------------------
Average Path Length: 3.339
Global Clustering Coefficient: 0.016
Network Metrics:
------------------------------
Average Path Length: 3.339
Global Clustering Coefficient: 0.016
Network Metrics:
------------------------------
Average Path Length: 3.339
Global Clustering Coefficient: 0.016
Network Metrics:
------------------------------
Average Path Length: 3.339
Global Clustering Coefficient: 0.016
Network Metrics:
------------------------------
Average Path Length: 3.339
Global Clustering Coefficient: 0.016
Network Metrics:
------------------------------
Average Path Length: 3.339
Global Clustering Coefficient: 0.016
Network Metrics:
------------------------------
Average Path Length: 3.339
Global Clustering Coefficient

False

In [30]:
import networkx as nx
import community.community_louvain as community
import numpy as np

def analyze_community_structure(G):
    # 1. Community Detection using python-louvain's best_partition
    partition = community.best_partition(G)

    # Convert partition dictionary to list of communities
    communities = {}
    for node, comm_id in partition.items():
        if comm_id not in communities:
            communities[comm_id] = []
        communities[comm_id].append(node)
    communities = list(communities.values())

    # 2. Calculate modularity
    modularity = community.modularity(partition, G)

    # 3. Calculate internal and external edges
    internal_edges = 0
    external_edges = 0
    total_edges = G.number_of_edges()

    for comm in communities:
        subgraph = G.subgraph(comm)
        internal_edges += subgraph.number_of_edges()
    external_edges = total_edges - internal_edges

    # 4. Calculate ratios
    internal_density = internal_edges / total_edges
    external_density = external_edges / total_edges

    print("Community Structure Analysis:")
    print("-" * 30)
    print(f"Number of communities: {len(communities)}")
    print(f"Modularity: {modularity:.3f}")
    print("\nEdge Distribution:")
    print(f"Internal edges: {internal_edges} ({internal_density:.1%})")
    print(f"External edges: {external_edges} ({external_density:.1%})")

    # 5. Interpretation
    is_community_based = (
        modularity > 0.3 and
        internal_density > 0.6 and
        len(communities) > 1
    )

    return is_community_based, partition

# To use it:
# result, partition = analyze_community_structure(G)

In [31]:
analyze_community_structure(G_morpho)

Community Structure Analysis:
------------------------------
Number of communities: 14
Modularity: 0.804

Edge Distribution:
Internal edges: 1214 (88.5%)
External edges: 158 (11.5%)


(True,
 {'Eos histrio': 0,
  'Trichoglossus euteles': 1,
  'Trichoglossus johnstoniae': 2,
  'Vini australis': 1,
  'Charmosyna palmarum': 2,
  'Prioniturus mada': 3,
  'Agapornis nigrigenis': 4,
  'Loriculus galgulus': 5,
  'Loriculus catamene': 2,
  'Nicopsitta calthrapae': 6,
  'Anodorhynchus hyacinthinus': 7,
  'Primolius maracana': 7,
  'Psittacara leucophthalmus': 8,
  'Aratinga jandaya': 0,
  'Pyrrhura cruentata': 6,
  'Pyrrhura frontalis': 6,
  'Pyrrhura hoematotis': 6,
  'Enicognathus ferrugineus': 0,
  'Pionites melanocephalus': 9,
  'Pionus chalcopterus': 9,
  'Pionus fuscus': 9,
  'Trichoglossus flavoviridis': 1,
  'Pionus tumultuosus': 9,
  'Coracopsis nigra': 10,
  'Psittacara rubritorquis': 8,
  'Aratinga maculata': 6,
  'Eupsittula astec': 6,
  'Geoffroyus hyacinthinus': 3,
  'Pyrrhura lucianii': 6,
  'Pyrrhura pacifica': 6,
  'Diopsittaca nobilis': 8,
  'Tanygnathus gramineus': 8,
  'Palaeornis eupatria': 8,
  'Trichoglossus weberi': 1,
  'Lorius garrulus': 11,
  'Caca

In [32]:
analyze_community_structure(G_rand)

Community Structure Analysis:
------------------------------
Number of communities: 11
Modularity: 0.362

Edge Distribution:
Internal edges: 637 (46.7%)
External edges: 728 (53.3%)


(False,
 {0: 9,
  1: 3,
  2: 10,
  3: 5,
  4: 7,
  5: 4,
  6: 1,
  7: 7,
  8: 7,
  9: 7,
  10: 4,
  11: 1,
  12: 9,
  13: 9,
  14: 10,
  15: 6,
  16: 7,
  17: 10,
  18: 0,
  19: 2,
  20: 5,
  21: 2,
  22: 3,
  23: 2,
  24: 6,
  25: 8,
  26: 9,
  27: 7,
  28: 3,
  29: 5,
  30: 3,
  31: 6,
  32: 3,
  33: 6,
  34: 0,
  35: 3,
  36: 2,
  37: 9,
  38: 3,
  39: 2,
  40: 2,
  41: 8,
  42: 9,
  43: 3,
  44: 1,
  45: 4,
  46: 7,
  47: 5,
  48: 9,
  49: 5,
  50: 7,
  51: 10,
  52: 5,
  53: 5,
  54: 9,
  55: 9,
  56: 1,
  57: 10,
  58: 1,
  59: 7,
  60: 3,
  61: 9,
  62: 7,
  63: 8,
  64: 2,
  65: 0,
  66: 5,
  67: 4,
  68: 8,
  69: 1,
  70: 2,
  71: 7,
  72: 10,
  73: 0,
  74: 3,
  75: 9,
  76: 1,
  77: 7,
  78: 2,
  79: 1,
  80: 9,
  81: 5,
  82: 4,
  83: 4,
  84: 7,
  85: 5,
  86: 2,
  87: 5,
  88: 8,
  89: 9,
  90: 8,
  91: 3,
  92: 8,
  93: 6,
  94: 3,
  95: 2,
  96: 8,
  97: 3,
  98: 9,
  99: 8,
  100: 1,
  101: 3,
  102: 5,
  103: 2,
  104: 1,
  105: 3,
  106: 0,
  107: 2,
  108: 7,
  109: